<a href="https://colab.research.google.com/github/ShovalBenjer/Natural_Language_Proccessing_NLP_Projects/blob/main/Trigram_Bigram_Unigram_English_Corpus_Wikipedia_484k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# -*- coding: utf-8 -*-
"""
N-gram Language Model Builder and Generator for Colab

Reads a text file, builds unigram, bigram, and trigram models,
calculates MLE probabilities, and generates new sentences based on the models.
"""

import re
import collections
from collections import Counter, defaultdict
import math
import random # Needed for weighted random choice

# --- Configuration ---
file_path = '/content/20-01.txt'
START_TOKEN = "<s>"
END_TOKEN = "</s>"
UNKNOWN_TOKEN = "<UNK>"

# --- 1. Text Preprocessing ---
# (Keep the preprocess_text function from the previous version)
def preprocess_text(filepath):
    """Reads a file, converts to lowercase, tokenizes, and adds sentence boundary markers."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            lines = f.readlines()
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        print("Please make sure you have uploaded the file to the /content/ directory.")
        return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None

    all_tokens = []
    print(f"Processing {len(lines)} lines...")
    processed_sentences = 0
    for i, line in enumerate(lines):
        try:
            line = re.sub(r'^@@\d+\s*<[ph]>\s*', '', line)
            line = re.sub(r'<[ph]>', ' ', line)
            line = re.sub(r'@@@@@@@@@@\s*', '', line)
            line = re.sub(r'https?\s*:\s*\*\*.*?\.\.\.', '', line)
            line = re.sub(r'\*\*.*?\.\.\.', '', line)
            line = line.lower()
            line = re.sub(r'[^\w\s\'\.\?\!]', '', line)
            sentences = re.split(r'(?<=[.?!])(?:\s+|$)', line.strip())
            for sentence in sentences:
                sentence = sentence.strip()
                if not sentence: continue
                words = re.findall(r'\b\w+\'?\w*\b', sentence)
                if not words: continue
                processed_sentences += 1
                tokens = [START_TOKEN, START_TOKEN] + words + [END_TOKEN]
                all_tokens.extend(tokens)
        except Exception as e:
            print(f"Error processing line {i+1}: {line[:100]}... Error: {e}")
            continue
        # if (i + 1) % 5000 == 0: print(f"Processed {i+1}/{len(lines)} lines...") # Reduce frequency if needed

    print(f"Found {processed_sentences} non-empty sentences.")
    print(f"Total tokens (including boundary markers): {len(all_tokens)}")
    if not all_tokens:
        print("Warning: No tokens were extracted.")
        return None
    if len(all_tokens) < 50: print("Sample tokens:", all_tokens)
    else: print("Sample tokens:", all_tokens[:25], "...", all_tokens[-25:])
    return all_tokens


# --- 2. N-gram Counting ---
# (Keep the build_ngram_counts function from the previous version)
def build_ngram_counts(tokens, n):
    """Builds counts for n-grams from a list of tokens."""
    if not tokens: return Counter()
    ngrams = []
    if len(tokens) >= n:
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i : i + n])
            ngrams.append(ngram)
    else: print(f"Warning: Not enough tokens ({len(tokens)}) to build any {n}-grams.")
    count_result = Counter(ngrams)
    if not count_result and len(tokens) >= n: print(f"Warning: No {n}-grams were generated despite sufficient tokens.")
    return count_result

# --- 3. Probability Calculation (Maximum Likelihood Estimation - MLE) ---
# (Keep the calculate_ngram_probabilities function from the previous version)
def calculate_ngram_probabilities(ngram_counts, n_minus_1_gram_counts, total_tokens_or_vocab_size, n):
    """Calculates MLE probabilities for n-grams."""
    probabilities = defaultdict(float)
    if not ngram_counts: return probabilities
    if n == 1:
        total_count = total_tokens_or_vocab_size
        if total_count == 0: return probabilities
        for word_tuple, count in ngram_counts.items():
            probabilities[word_tuple] = count / total_count
    elif n > 1:
        if not n_minus_1_gram_counts: return probabilities # Need prefixes
        zero_prefix_count = 0
        for ngram, count in ngram_counts.items():
            prefix = ngram[:-1]
            prefix_key = prefix[0] if n == 2 else prefix
            prefix_count = n_minus_1_gram_counts.get(prefix_key, 0)
            if prefix_count > 0:
                probabilities[ngram] = count / prefix_count
            else:
                probabilities[ngram] = 0.0
                zero_prefix_count += 1
        # if zero_prefix_count > 0: print(f"  Note: {zero_prefix_count} ngrams (n={n}) had zero-count prefixes.")
    else: print(f"Warning: Invalid n ({n}) in calculate_ngram_probabilities.")
    return probabilities

# --- 4. Sentence Generation Functions ---

def generate_unigram(unigram_probs, max_len=25):
    """Generates a sentence using the unigram model."""
    if not unigram_probs:
        return "(Error: Unigram probabilities unavailable)"

    # Get words and their probabilities, excluding boundary tokens for generation start
    # We will handle END_TOKEN explicitly during generation
    possible_words = [word[0] for word in unigram_probs.keys() if word[0] not in (START_TOKEN, END_TOKEN)]
    probabilities = [unigram_probs[ (word,) ] for word in possible_words]

    if not possible_words: # Handle case where only boundary tokens exist
         return "(Error: No valid words found in unigram model for generation)"

    # Normalize probabilities (important if they don't sum perfectly to 1 after filtering)
    prob_sum = sum(probabilities)
    if prob_sum <= 0:
         return "(Error: Cannot generate unigram, invalid probability sum)"
    probabilities = [p / prob_sum for p in probabilities]


    generated_sentence = []
    for _ in range(max_len):
        # Choose a word based on its overall probability
        chosen_word = random.choices(possible_words, weights=probabilities, k=1)[0]

        # Append the chosen word
        generated_sentence.append(chosen_word)

    # Join the words. Unigram doesn't naturally produce END_TOKEN based on context.
    return " ".join(generated_sentence)

def generate_bigram(bigram_probs, max_len=25):
    """Generates a sentence using the bigram model."""
    if not bigram_probs:
        return "(Error: Bigram probabilities unavailable)"

    current_word = START_TOKEN
    generated_sentence = []

    for _ in range(max_len):
        # Find possible next words based on the current word
        possible_next_options = {
            bg[1]: prob for bg, prob in bigram_probs.items() if bg[0] == current_word
        }

        if not possible_next_options:
            # Dead end - no known word follows the current one
            break

        next_words = list(possible_next_options.keys())
        probabilities = list(possible_next_options.values())

        # Normalize probabilities to ensure they sum to 1 for random.choices
        prob_sum = sum(probabilities)
        if prob_sum <= 0:
            break # Avoid division by zero or errors if probabilities are invalid
        probabilities = [p / prob_sum for p in probabilities]

        # Choose the next word based on conditional probabilities P(next | current)
        try:
             chosen_word = random.choices(next_words, weights=probabilities, k=1)[0]
        except ValueError:
             # Can happen if probabilities list is empty or contains invalid values
             print(f"  Warning (Bi): Value error during random choice. Context: {current_word}, Options: {next_words[:5]}, Probs: {probabilities[:5]}")
             break


        if chosen_word == END_TOKEN:
            break # Reached the end of the sentence

        generated_sentence.append(chosen_word)
        current_word = chosen_word # Update context

    return " ".join(generated_sentence) if generated_sentence else "(Empty bigram sentence)"


def generate_trigram(trigram_probs, max_len=25):
    """Generates a sentence using the trigram model."""
    if not trigram_probs:
        return "(Error: Trigram probabilities unavailable)"

    # Initial context is two start tokens
    context = (START_TOKEN, START_TOKEN)
    generated_sentence = []

    for _ in range(max_len):
        # Find possible next words based on the current context (last two words)
        possible_next_options = {
            tg[2]: prob for tg, prob in trigram_probs.items() if (tg[0], tg[1]) == context
        }

        if not possible_next_options:
            # Dead end - no known word follows the current context
            # Optionally, could implement backoff to bigram here
            break

        next_words = list(possible_next_options.keys())
        probabilities = list(possible_next_options.values())

        # Normalize probabilities
        prob_sum = sum(probabilities)
        if prob_sum <= 0:
            break
        probabilities = [p / prob_sum for p in probabilities]

        # Choose the next word based on conditional probabilities P(next | context)
        try:
            chosen_word = random.choices(next_words, weights=probabilities, k=1)[0]
        except ValueError:
            print(f"  Warning (Tri): Value error during random choice. Context: {context}, Options: {next_words[:5]}, Probs: {probabilities[:5]}")
            break


        if chosen_word == END_TOKEN:
            break # Reached the end of the sentence

        generated_sentence.append(chosen_word)
        # Update context: slide the window
        context = (context[1], chosen_word)

    return " ".join(generated_sentence) if generated_sentence else "(Empty trigram sentence)"


# --- Main Execution ---

print("--- Starting N-gram Model Building ---")

# 1. Preprocess Text
tokens = preprocess_text(file_path)

if tokens:
    # 2. Count N-grams
    print("\n--- Counting N-grams ---")
    unigram_counts = build_ngram_counts(tokens, 1)
    bigram_counts = build_ngram_counts(tokens, 2)
    trigram_counts = build_ngram_counts(tokens, 3)

    if not unigram_counts or not bigram_counts or not trigram_counts:
        print("\nError: Failed to generate n-gram counts. Cannot proceed.")
    else:
        print(f"Total Unigrams (tokens): {sum(unigram_counts.values())}")
        print(f"Unique Unigrams: {len(unigram_counts)}")
        # ... (rest of count printing) ...
        print(f"Total Trigrams: {sum(trigram_counts.values())}")
        print(f"Unique Trigrams: {len(trigram_counts)}")

        print("\n--- Sample N-gram Counts ---")
        if unigram_counts: print("Top 5 Unigrams:", unigram_counts.most_common(5))
        if bigram_counts: print("Top 5 Bigrams:", bigram_counts.most_common(5))
        if trigram_counts: print("Top 5 Trigrams:", trigram_counts.most_common(5))

        # 3. Calculate Probabilities (MLE)
        print("\n--- Calculating Probabilities (MLE) ---")
        total_token_count_for_unigrams = sum(unigram_counts.values())
        unigram_prefix_counts_for_bigrams = Counter(token for token in tokens) # Needed for bigram calculation context
        # Bigram counts are needed for trigram calculation context

        unigram_probs = calculate_ngram_probabilities(unigram_counts, None, total_token_count_for_unigrams, n=1)
        bigram_probs = calculate_ngram_probabilities(bigram_counts, unigram_prefix_counts_for_bigrams, None, n=2)
        trigram_probs = calculate_ngram_probabilities(trigram_counts, bigram_counts, None, n=3) # Use bigram_counts as context

        if not unigram_probs or not bigram_probs or not trigram_probs:
             print("\nError: Failed to calculate probabilities. Cannot proceed.")
        else:
            print("\n--- Sample N-gram Probabilities (MLE) ---")
            # (Keep the sample probability printing)
            # Unigrams
            print("Unigram Probabilities:")
            if unigram_counts:
              sample_unigrams = [ug for ug, count in unigram_counts.most_common(5)]
              for ug in sample_unigrams:
                  print(f"  P({ug[0]}) = {unigram_probs.get(ug, 0.0):.6f}")
            else: print("  No unigrams to calculate probabilities for.")

            # Bigrams
            print("\nBigram Probabilities:")
            if bigram_counts:
              sample_bigrams = [bg for bg, count in bigram_counts.most_common(5)]
              for bg in sample_bigrams:
                  prefix_uni = bg[0] # Prefix is a single token
                  print(f"  P({bg[1]} | {prefix_uni}) = {bigram_probs.get(bg, 0.0):.6f}  (Count({bg})={bigram_counts.get(bg,0)}, Count({prefix_uni})={unigram_prefix_counts_for_bigrams.get(prefix_uni, 0)})")
            else: print("  No bigrams to calculate probabilities for.")


            # Trigrams
            print("\nTrigram Probabilities:")
            if trigram_counts:
              sample_trigrams = [tg for tg, count in trigram_counts.most_common(5)]
              for tg in sample_trigrams:
                  prefix_bi = tg[:-1] # Prefix is a bigram tuple
                  print(f"  P({tg[2]} | {tg[0]}, {tg[1]}) = {trigram_probs.get(tg, 0.0):.6f}  (Count({tg})={trigram_counts.get(tg,0)}, Count({prefix_bi})={bigram_counts.get(prefix_bi, 0)})")
            else: print("  No trigrams to calculate probabilities for.")


            print("\n--- Model Building Complete ---")

            # ***** NEW: SENTENCE GENERATION SECTION *****
            print("\n\n--- Generating Sentences ---")
            num_sentences_to_generate = 7
            max_sentence_length = 20 # Limit length to avoid overly long/run-on sentences

            print(f"\n--- {num_sentences_to_generate} Sentences generated by Unigram Model ---")
            for i in range(num_sentences_to_generate):
                generated = generate_unigram(unigram_probs, max_len=max_sentence_length)
                print(f"{i+1}: {generated}")

            print(f"\n--- {num_sentences_to_generate} Sentences generated by Bigram Model ---")
            for i in range(num_sentences_to_generate):
                generated = generate_bigram(bigram_probs, max_len=max_sentence_length)
                print(f"{i+1}: {generated}")

            print(f"\n--- {num_sentences_to_generate} Sentences generated by Trigram Model ---")
            for i in range(num_sentences_to_generate):
                generated = generate_trigram(trigram_probs, max_len=max_sentence_length)
                print(f"{i+1}: {generated}")

            # Optional: Keep the user input loop for probability checking if desired
            # print("\n--- Calculate Probability for User Input ---")
            # while True:
            #     # ... (input loop code from previous answer) ...

else:
    print("\nCould not process the file or build models. Generation skipped.")

--- Starting N-gram Model Building ---
Processing 112 lines...
Found 3281 non-empty sentences.
Total tokens (including boundary markers): 75779
Sample tokens: ['<s>', '<s>', 'the', 'government', 'last', 'july', 'called', 'the', 'energy', 'sector', 'debt', 'situation', 'a', 'state', 'of', 'emergency', '</s>', '<s>', '<s>', 'this', 'was', 'during', 'the', 'midyear', 'budget'] ... ['your', 'hands', 'with', 'soap', 'and', 'water', 'cover', 'your', 'mouth', 'when', 'you', 'cough', 'or', 'sneeze', 'stay', 'home', 'from', 'work', 'until', 'you', 've', 'gone', 'one', 'day', '</s>']

--- Counting N-grams ---
Total Unigrams (tokens): 75779
Unique Unigrams: 9692
Total Trigrams: 75777
Unique Trigrams: 60325

--- Sample N-gram Counts ---
Top 5 Unigrams: [(('<s>',), 6562), (('the',), 4050), (('</s>',), 3281), (('to',), 1877), (('of',), 1775)]
Top 5 Bigrams: [(('<s>', '<s>'), 3281), (('</s>', '<s>'), 3280), (('of', 'the'), 442), (('<s>', 'the'), 418), (('in', 'the'), 341)]
Top 5 Trigrams: [(('</s>', 